In [ ]:
import pandas as pd
from autogluon.tabular import TabularPredictor
import mlflow
import os

In [11]:
X_train = pd.read_csv('../data/processed/X_train.csv')
y_train = pd.read_csv('../data/processed/y_train.csv')

y_train = y_train.squeeze() # converts series to dataframe with one column
train_data = X_train.copy()
train_data['churn'] = y_train

train_data.to_csv('../data/processed/train_data.csv', index=False)

In [12]:
X_test = pd.read_csv('../data/processed/X_test.csv')
y_test = pd.read_csv('../data/processed/y_test.csv')
y_test = y_test.squeeze()
test_data = X_test.copy()
test_data['churn'] = y_test
test_data.to_csv('../data/processed/test_data.csv')

In [ ]:
label = 'churn'

mlflow.set_experiment('churn-prediction-automl')

with mlflow.start_run(run_name="AutoGluon_Classification"):
    # train autoML model
    predictor = TabularPredictor(label=label).fit(train_data, time_limit=3600)

    # evaluate on test set
    perf = predictor.evaluate(test_data)

    # log leaderboard as csv
    leaderboard = predictor.leaderboard(silent=True)
    leaderboard.to_csv("automl_leaderboard.csv", index=False)
    mlflow.log_artifact("automl_leaderboard.csv")

    # Save predictor to a deterministic absolute path and log directory
    predictor_dir = os.path.abspath("./autogluon_predictor")
    os.makedirs(predictor_dir, exist_ok=True)
    predictor.save(predictor_dir)
    save_dir = predictor.path
    assert isinstance(save_dir, str) and os.path.isdir(save_dir)
    mlflow.log_artifacts(save_dir)

    # log summary metric (accuracy)
    accuracy = perf.get('accuracy', None)
    if accuracy is not None:
        mlflow.log_metric("accuracy", accuracy)

    print("AutoML training completed and logged in MLflow")

No path specified. Models will be saved in: "AutogluonModels/ag-20251105_090618"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Fri May  2 14:22:13 UTC 2025
CPU Count:          8
Memory Avail:       2.54 GB / 11.47 GB (22.1%)
Disk Space Avail:   27.05 GB / 236.42 GB (11.4%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU mem

AssertionError: 